# Обучение модели

## 1. Загрузка библиотек

In [1]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import statistics
from scipy import stats

import os

import pandas as pd
import numpy as np
import random
from scipy import stats
import math
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error,  accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression, load_boston, load_iris
from sklearn import linear_model

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, \
StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet,\
RidgeClassifier, LogisticRegressionCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score, accuracy_score, \
roc_auc_score, classification_report, r2_score, precision_score, recall_score, \
log_loss, mean_squared_log_error

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz 
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve

from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, \
    f1_score, log_loss, classification_report, roc_curve, auc, roc_auc_score, r2_score, \
    mean_squared_log_error

from tqdm.notebook import tqdm

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore")
simplefilter("ignore", category=RuntimeWarning)

RAND = 10
N_FOLDS = 5

ModuleNotFoundError: No module named 'xgboost'

## 2. Загрузка подготовленного датасета

In [ ]:
df = pd.read_csv('df_train.csv', sep=',')

## 3. Catboost

### 3.1. Catboost baseline

In [ ]:
X = df.drop(['Sample_ID', "Label"], axis=1)
y = df["Label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=RAND)

X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)

In [ ]:
cat_features = X_val.select_dtypes('category').columns.tolist()

clfcat = CatBoostClassifier(scale_pos_weight=y.agg(lambda x: x.eq(0).sum()) /
                            y.agg(lambda x: x.eq(1).sum()),
                            random_state=RAND,
                            eval_metric="F1",
                            cat_features=cat_features)
eval_set = [(X_val, y_val)]

clfcat.fit(X_train_,
           y_train_,
           eval_set=eval_set,
           early_stopping_rounds=100,
           verbose=False)

y_pred_cat = clfcat.predict(X_test)

In [ ]:
def get_metrics(y_test, y_pred, y_score, name):
    df_metrics = pd.DataFrame()
    
    df_metrics['model'] = [name]
    
    # Основные метрики для задачи классификации
    df_metrics['Accuracy'] = [accuracy_score(y_test, y_pred)]
    
    # В ROC-AUC подаем фактические значения y и вероятности!!!!
    df_metrics['ROC_AUC'] = [roc_auc_score(y_test, y_score[:,1])]
    df_metrics['Precision'] = [precision_score(y_test, y_pred)]
    df_metrics['Recall'] = [recall_score(y_test, y_pred)]
    df_metrics['f1'] = [f1_score(y_test, y_pred)]
    df_metrics['Logloss'] = [log_loss(y_test, y_score)]
    
    return df_metrics

In [ ]:
y_score = clfcat.predict_proba(X_test)

metrics = get_metrics(y_test,
                      y_pred_cat,
                      y_score,
                      name='Catboost_baseline_test')
metrics

In [ ]:
y_score_train = clfcat.predict_proba(X_train)
y_pred_cat_train = clfcat.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='Catboost_baseline_train'))

metrics

In [ ]:
"""
Underfitting, берем f1 за eval_metric и loss function! =)

и далее так везде
"""

### 3.2. Catboost optuna

In [ ]:
from catboost import Pool

cat_feat = X_train.select_dtypes('category').columns.tolist()


def objective_cat_first(trial, X, y, N_FOLDS, random_state, cat_feat):
    params = {
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate",
                                             0.001,
                                             0.5,
                                             log=True),
        "random_state": RAND
    }

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        train_data = Pool(data=X_train, label=y_train, cat_features=cat_feat)
        eval_data = Pool(data=X_test, label=y_test, cat_features=cat_feat)

        model = CatBoostClassifier(**params)
        model.fit(train_data,
                  eval_set=eval_data,
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
import optuna

study_cat = optuna.create_study(direction="maximize", study_name="Cat_02")
func = lambda trial: objective_cat_first(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND, cat_feat=cat_feat)
# n_trials - кол-во итераций
study_cat.optimize(func, n_trials=10, show_progress_bar=True)

In [ ]:
def objective_cat_second(trial, X, y, N_FOLDS, random_state, cat_feat):
    params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_categorical("learning_rate",
                                  [0.09121441559229158]),  # 0.07
        "max_depth":
        trial.suggest_int("max_depth", 3, 12),
        "l2_leaf_reg":
        trial.suggest_uniform("l2_leaf_reg", 1e-5, 1e2),
        "random_strength":
        trial.suggest_uniform("random_strength", 10, 50),
        "border_count":
        trial.suggest_categorical("border_count", [128, 254]),
        # "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS", "No"]),
        # "grow_policy":
        # trial.suggest_categorical("grow_policy",
        #                        ["SymmetricTree", "Depthwise", "Lossguide"]),
        # "od_wait":
        # trial.suggest_int("od_wait", 500, 2000),
        # "leaf_estimation_iterations":
        #trial.suggest_int('leaf_estimation_iterations', 1, 15),
        # "cat_features": trial.suggest_categorical("cat_features", ["cat_features"]),
        "use_best_model":
        trial.suggest_categorical("use_best_model", [True]),
         "eval_metric":
         trial.suggest_categorical("eval_metric", ["F1"]),
        "loss_function":
        trial.suggest_categorical("loss_function", ["Logloss"]),
        # "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        # "class_weights": trial.suggest_categorical("class_weights", [1]),
        "scale_pos_weight":
        trial.suggest_categorical(
            "scale_pos_weight",
            [y.agg(lambda x: x.eq(0).sum()) / y.agg(lambda x: x.eq(1).sum())]),
        "random_state":
        RAND
    }

    # if params["bootstrap_type"] == "Bayesian":
    #  params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    # elif params["bootstrap_type"] == "Bernoulli":
    #  params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    # if params["grow_policy"] == "Lossguide":
    # params["num_leaves"] = trial.suggest_int("num_leaves", 20, 1000)

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        train_data = Pool(data=X_train, label=y_train, cat_features=cat_feat)
        eval_data = Pool(data=X_test, label=y_test, cat_features=cat_feat)

        model = CatBoostClassifier(**params)
        model.fit(train_data,
                  eval_set=eval_data,
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
study_cat = optuna.create_study(direction="maximize", study_name="Cat_02")
func = lambda trial: objective_cat_second(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND, cat_feat=cat_feat)
# n_trials - кол-во итераций
study_cat.optimize(func, n_trials=10, show_progress_bar=True)

In [ ]:
study_cat.best_params

In [ ]:
cat_optuna = CatBoostClassifier(**study_cat.best_params)
cat_optuna.fit(X_train_,
               y_train_,
               cat_features=cat_features,
               eval_set=eval_set,
               verbose=False,
               early_stopping_rounds=100)

y_pred = cat_optuna.predict(X_test)
y_score = cat_optuna.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=y_pred,
                y_score=y_score,
                name='CatBoost_Optuna_test'))

metrics

In [ ]:
y_score_train = cat_optuna.predict_proba(X_train)
y_pred_cat_train = cat_optuna.predict(X_train)

metrics =metrics.append(get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='CatBoost_Optuna_train'))

metrics

### 3.3. Catboost holdout

In [ ]:
"""
это уже фунциями для стекинга =)
это не holdout

Я забрал его прям с лекции по бустингам, там не было стекинга...
Просто добавил вконце 5 строк, чтобы параллельно с обычными моделями, строилась таблица для стекинга.
Поправьте, пожалуйста, если ошибаюсь)


в этой функции тут смысла нет, если просто holdout, лучше без неее
но вы же помои хотите в стегинге применять, лучше написать это словами

НО, я увидела ниже, что далее функцию не применяете у другим алгоритмам, убираем ее и делаем как ниже у вас
"""

In [ ]:
meta_X = pd.DataFrame()
meta_X_test = pd.DataFrame()

cv = StratifiedKFold(n_splits=N_FOLDS)

score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[train_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[train_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = CatBoostClassifier(**study_cat.best_params)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                silent=True,
                cat_features=cat_features,
                early_stopping_rounds=100)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
meta_X['cat_hold'] = model.predict_proba(X_train)[:,1]
meta_X_test['cat_hold'] = model.predict_proba(X_test)[:,1]

In [ ]:
meta_X

In [ ]:
meta_X_test

In [ ]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)

In [ ]:
from scipy import stats

test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

# oof - out-of-fold  - предсказания/значение метрики при обучении модели
# в ходе перекрестной проверки на validation data
print('F1 mean OOF: %.3f, std: %.3f' %
      (np.mean(score_oof), np.std(score_oof)))

# holdout  - предсказания/значение метрики при обучении модели
# в ходе перекрестной проверки на holdout data
print('F1 HOLDOUT: %.3f' % f1_score(y_test, test_pred))

In [ ]:
metrics=metrics.append(get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='CatBoost_Optuna_Holdout_test'))

metrics

In [ ]:
train_pred = model.predict(X_train)
train_pred_proba = model.predict_proba(X_train)

metrics=metrics.append(get_metrics(y_test=y_train,
                y_pred=train_pred,
                y_score=train_pred_proba,
                name='CatBoost_Optuna_Holdout_train'))

metrics

## 4. RandomForest holdout+optuna

In [ ]:
rf = RandomForestClassifier(random_state=RAND, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
X_bin = pd.get_dummies(X, drop_first=True)
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=RAND)



def objective_rf(trial, X, y, N_FOLDS, rand):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "criterion": trial.suggest_categorical("criterion", ['gini', 'entropy', 'log_loss']),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        'bootstrap': trial.suggest_categorical("bootstrap", [True]),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'max_samples': trial.suggest_float('max_samples', 0.3, 0.99),
        'max_features': trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2']),
        'class_weight': trial.suggest_categorical("class_weight", ['balanced']),
        #'random_state': [rand] 
    }
    
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=rand)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = RandomForestClassifier(**params, n_jobs=-1, random_state=rand)
        model.fit(X_train, y_train)
        
        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
study_rf = optuna.create_study(direction="maximize", study_name="RF_01")
func = lambda trial: objective_rf(trial, X_train_bin, y_train, N_FOLDS, rand=RAND)
# n_trials - кол-во итераций
study_rf.optimize(func, n_trials=20, show_progress_bar=True)

In [ ]:
"""
А тут уже функцию ранее не используете =) как то не логично
давайте убирать ту функцию выше, делайте для катбуста как тут
"""

pred_val = []
pred_score_val = []


cv = StratifiedKFold(n_splits=N_FOLDS)

for fold, (train_index, test_index) in enumerate(cv.split(X_train_bin, y_train)):
    X_train_, X_val = X_train_bin.iloc[train_index], X_train_bin.iloc[test_index]
    y_train_, y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    model = RandomForestClassifier(**study_rf.best_params, n_jobs=-1)
    model.fit(X_train_, y_train_)

    y_pred_val = model.predict(X_val)
    y_score_val = model.predict_proba(X_val)

    print("Fold:", fold + 1,
          "F1 SCORE %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # holdout list
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)

model.fit(X_train_bin, y_train)

meta_X['rf_01'] = np.concatenate(pred_score_val)[:,1]
meta_X_test['rf_01'] = model.predict_proba(X_test_bin)[:,1]

In [ ]:
meta_X_test

In [ ]:
y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)

metrics = metrics.append(get_metrics(y_test, y_pred, y_score, 
                                     name='RandomForestClassifier_test'))
metrics

In [ ]:
y_score_train = rf.predict_proba(X_train)
y_pred_cat_train = rf.predict(X_train)

metrics =metrics.append(get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='RandomForestClassifier_train'))

metrics

In [ ]:
meta_X_test

## 5. LGBM

### 5.1. LGBM baseline

In [ ]:
clfLGBM = LGBMClassifier(scale_pos_weight=3, random_state=RAND)
eval_set = [(X_val, y_val)]

clfLGBM.fit(X_train_,
        y_train_,
        eval_metric="F1",
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=2)

y_pred = clfLGBM.predict(X_test)

In [ ]:
y_pred = clfLGBM.predict(X_test)
y_score = clfLGBM.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=y_pred,
                y_score=y_score,
                name='LGBMboost_eval_baseline_test'))

metrics

In [ ]:
y_score_train = clfLGBM.predict_proba(X_train)
y_pred_cat_train = clfLGBM.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBMboost_eval_baseline_train'))

metrics

### 5.2. LGBM Optuna

In [ ]:
def objective_lgb_first(trial, X, y, N_FOLDS, random_state=10):
    lgb_params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
    }

    N_FOLDS = 3
    cv = KFold(n_splits=N_FOLDS, shuffle=True)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # стрижка "на лету"
        # observation_key - Оценочная метрика для обрезки
        model = LGBMClassifier(**lgb_params)
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric="F1",
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGB_21")


def func(trial): return objective_lgb_first(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND)


# n_trials - кол-во итераций
study.optimize(func, n_trials=20, show_progress_bar=True)

In [ ]:
def objective_lgb_second(trial, X, y, N_FOLDS, random_state=10):
    lgb_params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_categorical("learning_rate", [0.013607708322422411]),
        "num_leaves":
        trial.suggest_int("num_leaves", 20, 1000, step=20),
        "max_depth":
        trial.suggest_int("max_depth", 3, 12),
        "lambda_l1":
        trial.suggest_int("lambda_l1", 0, 100),
        "lambda_l2":
        trial.suggest_int("lambda_l2", 0, 100),
        "min_gain_to_split":
        trial.suggest_int("min_gain_to_split", 0, 20),
        "bagging_fraction":
        trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "bagging_freq":
        trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction":
        trial.suggest_float("feature_fraction", 0.2, 1.0),
        # "objective": trial.suggest_categorical("objective", ["mae"]),
        "random_state":
        RAND
    }

    N_FOLDS = 3
    cv = KFold(n_splits=N_FOLDS, shuffle=True)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # стрижка "на лету"
        # observation_key - Оценочная метрика для обрезки
        pruning_callback = optuna.integration.LightGBMPruningCallback(
            trial, "F1")
        model = LGBMClassifier(**lgb_params)
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric="F1",
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGB_22")
func = lambda trial: objective_lgb_second(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND)
# n_trials - кол-во итераций
study.optimize(func, n_trials=20, show_progress_bar=True)

In [ ]:
lgb_grid = LGBMClassifier(**study.best_params)
lgb_grid.fit(X_train_,
             y_train_,
             eval_metric="F1",
             eval_set=eval_set,
             verbose=False,
             early_stopping_rounds=100)

y_pred = lgb_grid.predict(X_test)
y_score = lgb_grid.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test.values,
                y_pred=y_pred,
                y_score=y_score,
                name='LGBoost_Optuna_test'))
metrics

In [ ]:
y_score_train = lgb_grid.predict_proba(X_train)
y_pred_cat_train = lgb_grid.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBoost_Optuna_train'))

metrics

### 5.3. LGBM Holdout

In [ ]:
pred_val = []
pred_score_val = []

cv = StratifiedKFold(n_splits=N_FOLDS)
for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

    model = LGBMClassifier(is_unbalance=True, random_state=RAND)
    model.fit(X_train_,
              y_train_,
              eval_set=[(X_val, y_val)],
              eval_metric="F1",
              early_stopping_rounds=100,
              verbose=0)

    y_pred_val = model.predict(X_val)
    y_score_val = model.predict_proba(X_val)

    print("Fold:", fold + 1,
          "F1 SCORE %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # holdout list
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)

model.fit(X_train, y_train)

meta_X['lgb_01'] = np.concatenate(pred_score_val)[:, 1]
meta_X_test['lgb_01'] = model.predict_proba(X_test)[:, 1]

In [ ]:
score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[train_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[train_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = LGBMClassifier(**study.best_params)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                eval_metric="F1",
                early_stopping_rounds=100,
                verbose=0)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
    
meta_X['lgb_02'] = model.predict_proba(X_train)[:, 1]
meta_X_test['lgb_02'] = model.predict_proba(X_test)[:, 1]

In [ ]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)
test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

In [ ]:
meta_X

In [ ]:
meta_X_test

In [ ]:
metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='LGBM_Holdout_test'))

metrics

In [ ]:
y_score_train = model.predict_proba(X_train)
y_pred_cat_train = model.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBM_Holdout_train'))

metrics

## 6. xgboost holdout

In [ ]:
import xgboost as xgb

score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train_bin, y_train)):
    X_train_, X_val = X_train_bin.iloc[train_idx], X_train_bin.iloc[test_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = XGBClassifier(scale_pos_weight=3, random_state=RAND)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                eval_metric="logloss",
                early_stopping_rounds=100,
                verbose=0)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
    
meta_X['xgb_01'] = model.predict_proba(X_train)[:, 1]
meta_X_test['xgb_01'] = model.predict_proba(X_test)[:, 1]

In [ ]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)
test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

In [ ]:
metrics=metrics.append(get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='XGB_Holdout_test'))

metrics

In [ ]:
y_score_train = model.predict_proba(X_train)
y_pred_cat_train = model.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='XGB_Holdout_train'))

metrics

## 7. Stacking

In [ ]:
meta_X_test

In [ ]:
meta_X

In [ ]:
df_label_train = pd.DataFrame(data=y_train)
df_label_train = df_label_train.reset_index()
meta_X_final=pd.concat([meta_X, df_label_train], axis=1)
meta_X_final.drop(['index'], axis=1, inplace=True)
meta_X_final

In [ ]:
y_train_final = meta_X_final['Label']
X_train_final = meta_X_final.drop(['Label'], axis=1)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_train_final,
                                                            y_train_final,
                                                            test_size=0.2,
                                                            stratify=y_train,
                                                            random_state=RAND)

X_train_, X_val, y_train_, y_val = train_test_split(X_train_f,
                                                    y_train_f,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)

clfcat = CatBoostClassifier(scale_pos_weight=3,
                            random_state=RAND,
                            eval_metric="F1")
eval_set = [(X_val, y_val)]

clfcat.fit(X_train_,
           y_train_,
           eval_set=eval_set,
           early_stopping_rounds=100,
           verbose=False)

y_pred = clfcat.predict(meta_X_test)

In [ ]:
y_score = clfcat.predict_proba(meta_X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test, y_pred=y_pred, y_score=y_score,
                name='Stacking_test'))

metrics

In [ ]:
y_score = clfcat.predict_proba(meta_X)
y_pred = clfcat.predict(meta_X)

metrics = metrics.append(
    get_metrics(y_test=y_train, y_pred=y_pred, y_score=y_score,
                name='Stacking_train'))

metrics

Выводы:
- 1) Лучшие показатели f1, roc_auc, logloss на тестовых данных выдал CatBoost_Optuna_Holdout 
- 2) Самые непереобученные модели: LGBoost_Optuna, LGBM_Holdout.
- 3) Stacking не показал хороших результатов на тестовых данных, но показал лучшие результаты, среди моих моделей, на скрытых данных, по которым и ставится оценка в соревновании
- 4) RandomForest показал одни и самых плохих результатов по основным метрикам, но лучший по Precision. Поэтому я оставил его в стэкинге.
- 5) Пробовал разное кол-во и состав моделей для стекинга. Самые лучшие метрики на скрытых данных получились у этих 5 моделей.
- 6) Метрики на тестовых данных и на скрытых данных не коррелировали между собой, поэтому обучение для соревнований проходило вслепую.

[11 место в соревновании](https://zindi.africa/competitions/landslide-prevention-and-innovation-challenge/leaderboard)